In [6]:
!pip install pandas requests



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import necessary libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Import API key from config.py
from api_key import API_KEY

BASE_URL = 'https://data.nasdaq.com/api/v3/datasets/WIKI/'

# Function to fetch data from NASDAQ Data Link API
def fetch_data(dataset_code, api_key):
    url = f"{BASE_URL}{dataset_code}.json"
    params = {'api_key': api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# Fetch data for NVIDIA, Google, and Apple
dataset_codes = ['NVDA', 'GOOGL', 'AAPL']
data = {code: fetch_data(code, API_KEY) for code in dataset_codes}

# Create DataFrames from the data
dfs = {}
for code in dataset_codes:
    column_names = data[code]['dataset']['column_names']
    rows = data[code]['dataset']['data']
    df = pd.DataFrame(rows, columns=column_names)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    dfs[code] = df

# Display the DataFrames
for code, df in dfs.items():
    print(f"Data for {code}")
    display(df.head())

# Create visualizations using Matplotlib



Data for NVDA


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-27,247.75,250.00,219.845,225.52,34462113.0,0.0,1.0,247.75,250.00,219.845,225.52,34462113.0
2018-03-26,238.00,244.53,235.900,244.48,15130542.0,0.0,1.0,238.00,244.53,235.900,244.48,15130542.0
2018-03-23,242.40,242.67,232.520,232.97,18225390.0,0.0,1.0,242.40,242.67,232.520,232.97,18225390.0
2018-03-22,246.00,247.88,240.341,241.85,13663927.0,0.0,1.0,246.00,247.88,240.341,241.85,13663927.0
2018-03-21,249.32,252.00,247.330,248.56,10841782.0,0.0,1.0,249.32,252.00,247.330,248.56,10841782.0


Data for GOOGL


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0,0.0,1.0,1063.90,1064.54,997.62,1006.94,2940957.0
2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0,0.0,1.0,1050.60,1059.27,1010.58,1054.09,3272409.0
2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0,0.0,1.0,1051.37,1066.78,1024.87,1026.55,2413517.0
2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0,0.0,1.0,1080.01,1083.92,1049.64,1053.15,3418154.0
2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0,0.0,1.0,1092.57,1108.70,1087.21,1094.00,1990515.0


Data for AAPL


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-27,173.68,175.15,166.92,168.340,38962839.0,0.0,1.0,173.68,175.15,166.92,168.340,38962839.0
2018-03-26,168.07,173.10,166.44,172.770,36272617.0,0.0,1.0,168.07,173.10,166.44,172.770,36272617.0
2018-03-23,168.39,169.92,164.94,164.940,40248954.0,0.0,1.0,168.39,169.92,164.94,164.940,40248954.0
2018-03-22,170.00,172.68,168.60,168.845,41051076.0,0.0,1.0,170.00,172.68,168.60,168.845,41051076.0
2018-03-21,175.04,175.09,171.26,171.270,35247358.0,0.0,1.0,175.04,175.09,171.26,171.270,35247358.0
